# ध्यान तंत्र और ट्रांसफॉर्मर

पुनरावर्ती नेटवर्क्स (Recurrent Networks) की एक बड़ी कमी यह है कि किसी अनुक्रम (sequence) के सभी शब्दों का परिणाम पर समान प्रभाव होता है। यह समस्या नामित इकाई पहचान (Named Entity Recognition) और मशीन अनुवाद (Machine Translation) जैसे अनुक्रम-से-अनुक्रम कार्यों के लिए मानक LSTM एन्कोडर-डिकोडर मॉडल्स के प्रदर्शन को कम कर देती है। वास्तविकता में, इनपुट अनुक्रम के कुछ विशेष शब्दों का अनुक्रमिक आउटपुट पर अन्य शब्दों की तुलना में अधिक प्रभाव होता है।

मान लीजिए कि हमारे पास एक अनुक्रम-से-अनुक्रम मॉडल है, जैसे मशीन अनुवाद। इसे दो पुनरावर्ती नेटवर्क्स द्वारा लागू किया जाता है, जहां एक नेटवर्क (**एन्कोडर**) इनपुट अनुक्रम को छिपी हुई अवस्था (hidden state) में संकुचित करता है, और दूसरा नेटवर्क (**डिकोडर**) इस छिपी हुई अवस्था को अनुवादित परिणाम में विस्तारित करता है। इस दृष्टिकोण की समस्या यह है कि नेटवर्क की अंतिम अवस्था वाक्य की शुरुआत को याद रखने में कठिनाई महसूस करती है, जिससे लंबे वाक्यों पर मॉडल की गुणवत्ता खराब हो जाती है।

**ध्यान तंत्र (Attention Mechanisms)** प्रत्येक इनपुट वेक्टर के संदर्भीय प्रभाव को RNN के प्रत्येक आउटपुट भविष्यवाणी पर भारित करने का एक साधन प्रदान करते हैं। इसे लागू करने का तरीका यह है कि इनपुट RNN की मध्यवर्ती अवस्थाओं और आउटपुट RNN के बीच शॉर्टकट्स बनाए जाते हैं। इस प्रकार, जब आउटपुट प्रतीक $y_t$ उत्पन्न किया जाता है, तो हम सभी इनपुट छिपी अवस्थाओं $h_i$ को विभिन्न भार गुणांक $\alpha_{t,i}$ के साथ ध्यान में रखेंगे।

![एन्कोडर/डिकोडर मॉडल का चित्र जिसमें एक एडिटिव ध्यान परत है](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*एन्कोडर-डिकोडर मॉडल एडिटिव ध्यान तंत्र के साथ [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) से, [इस ब्लॉग पोस्ट](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) से उद्धृत*

ध्यान मैट्रिक्स $\{\alpha_{i,j}\}$ यह दर्शाता है कि आउटपुट अनुक्रम में किसी दिए गए शब्द को उत्पन्न करने में कौन से इनपुट शब्द कितनी भूमिका निभाते हैं। नीचे इस प्रकार की एक मैट्रिक्स का उदाहरण दिया गया है:

![चित्र जिसमें RNNsearch-50 द्वारा पाए गए एक नमूना संरेखण को दिखाया गया है, Bahdanau - arviz.org से लिया गया](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3) से लिया गया चित्र]*

ध्यान तंत्र वर्तमान या लगभग वर्तमान प्राकृतिक भाषा प्रसंस्करण (Natural Language Processing) में अत्याधुनिक तकनीकों के लिए जिम्मेदार हैं। हालांकि, ध्यान जोड़ने से मॉडल के पैरामीटरों की संख्या में काफी वृद्धि होती है, जिससे RNNs के साथ स्केलिंग समस्याएं उत्पन्न होती हैं। RNNs को स्केल करने की एक प्रमुख बाधा यह है कि मॉडल की पुनरावर्ती प्रकृति प्रशिक्षण को बैच और समानांतर बनाने में चुनौतीपूर्ण बनाती है। RNN में अनुक्रम के प्रत्येक तत्व को क्रमिक रूप से संसाधित करना पड़ता है, जिससे इसे आसानी से समानांतर नहीं किया जा सकता।

ध्यान तंत्रों को अपनाने और इस बाधा ने आज के अत्याधुनिक ट्रांसफॉर्मर मॉडल्स के निर्माण का मार्ग प्रशस्त किया, जिन्हें हम BERT से OpenGPT3 तक जानते और उपयोग करते हैं।

## ट्रांसफॉर्मर मॉडल्स

प्रत्येक पूर्वानुमान के संदर्भ को अगले मूल्यांकन चरण में अग्रेषित करने के बजाय, **ट्रांसफॉर्मर मॉडल्स** **स्थिति एन्कोडिंग्स (positional encodings)** और ध्यान का उपयोग करके दिए गए इनपुट के संदर्भ को एक निर्दिष्ट पाठ विंडो के भीतर कैप्चर करते हैं। नीचे दी गई छवि दिखाती है कि स्थिति एन्कोडिंग्स और ध्यान का उपयोग करके किसी विंडो के भीतर संदर्भ को कैसे कैप्चर किया जा सकता है।

![एनिमेटेड GIF जो दिखाता है कि ट्रांसफॉर्मर मॉडल्स में मूल्यांकन कैसे किया जाता है।](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

चूंकि प्रत्येक इनपुट स्थिति को स्वतंत्र रूप से प्रत्येक आउटपुट स्थिति पर मैप किया जाता है, ट्रांसफॉर्मर RNNs की तुलना में बेहतर समानांतरता प्रदान करते हैं, जिससे बहुत बड़े और अधिक अभिव्यक्तिपूर्ण भाषा मॉडल्स सक्षम होते हैं। प्रत्येक ध्यान हेड का उपयोग शब्दों के बीच विभिन्न संबंधों को सीखने के लिए किया जा सकता है, जो डाउनस्ट्रीम प्राकृतिक भाषा प्रसंस्करण कार्यों को बेहतर बनाता है।

**BERT** (Bidirectional Encoder Representations from Transformers) एक बहुत बड़ा बहु-स्तरीय ट्रांसफॉर्मर नेटवर्क है, जिसमें *BERT-base* के लिए 12 परतें और *BERT-large* के लिए 24 परतें होती हैं। इस मॉडल को पहले बड़े पाठ डेटा (विकिपीडिया + किताबें) पर असुपरवाइज्ड प्रशिक्षण (वाक्य में छिपे हुए शब्दों की भविष्यवाणी) का उपयोग करके प्रशिक्षित किया जाता है। पूर्व-प्रशिक्षण के दौरान, मॉडल भाषा की महत्वपूर्ण समझ को आत्मसात करता है, जिसे फिर अन्य डेटासेट्स के साथ फाइन-ट्यूनिंग के माध्यम से उपयोग किया जा सकता है। इस प्रक्रिया को **ट्रांसफर लर्निंग** कहा जाता है।

![चित्र http://jalammar.github.io/illustrated-bert/ से](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

ट्रांसफॉर्मर आर्किटेक्चर के कई प्रकार हैं, जैसे BERT, DistilBERT, BigBird, OpenGPT3 और अन्य, जिन्हें फाइन-ट्यून किया जा सकता है। [HuggingFace पैकेज](https://github.com/huggingface/) PyTorch के साथ इनमें से कई आर्किटेक्चर को प्रशिक्षित करने के लिए एक रिपॉजिटरी प्रदान करता है।

## टेक्स्ट वर्गीकरण के लिए BERT का उपयोग

आइए देखें कि हम अपने पारंपरिक कार्य को हल करने के लिए पूर्व-प्रशिक्षित BERT मॉडल का उपयोग कैसे कर सकते हैं: अनुक्रम वर्गीकरण। हम अपने मूल AG News डेटासेट को वर्गीकृत करेंगे।

सबसे पहले, HuggingFace लाइब्रेरी और हमारा डेटासेट लोड करें:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


क्योंकि हम प्री-ट्रेंड BERT मॉडल का उपयोग करेंगे, हमें एक विशेष टोकनाइज़र का उपयोग करना होगा। सबसे पहले, हम प्री-ट्रेंड BERT मॉडल से जुड़े टोकनाइज़र को लोड करेंगे।

HuggingFace लाइब्रेरी में प्री-ट्रेंड मॉडल्स का एक रिपॉजिटरी है, जिसे आप केवल उनके नामों को `from_pretrained` फंक्शन के आर्ग्युमेंट्स के रूप में देकर उपयोग कर सकते हैं। मॉडल के लिए आवश्यक सभी बाइनरी फाइलें स्वचालित रूप से डाउनलोड हो जाएंगी।

हालांकि, कुछ स्थितियों में आपको अपने खुद के मॉडल्स लोड करने की आवश्यकता हो सकती है। ऐसे मामलों में, आप उस डायरेक्टरी को निर्दिष्ट कर सकते हैं जिसमें सभी संबंधित फाइलें हों, जैसे टोकनाइज़र के लिए पैरामीटर्स, मॉडल पैरामीटर्स के साथ `config.json` फाइल, बाइनरी वेट्स आदि।


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` ऑब्जेक्ट में `encode` फ़ंक्शन होता है जिसे सीधे टेक्स्ट को एन्कोड करने के लिए उपयोग किया जा सकता है:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

फिर, आइए ऐसे इटरेटर बनाते हैं जिन्हें हम प्रशिक्षण के दौरान डेटा तक पहुंचने के लिए उपयोग करेंगे। क्योंकि BERT अपनी स्वयं की एनकोडिंग फ़ंक्शन का उपयोग करता है, हमें एक पैडिंग फ़ंक्शन को परिभाषित करने की आवश्यकता होगी जो पहले परिभाषित `padify` के समान हो:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

हमारे मामले में, हम पूर्व-प्रशिक्षित BERT मॉडल का उपयोग करेंगे जिसे `bert-base-uncased` कहा जाता है। आइए मॉडल को `BertForSequenceClassfication` पैकेज का उपयोग करके लोड करें। यह सुनिश्चित करता है कि हमारे मॉडल में पहले से ही वर्गीकरण के लिए आवश्यक संरचना है, जिसमें अंतिम वर्गीकर्ता भी शामिल है। आपको एक चेतावनी संदेश दिखाई देगा जिसमें कहा जाएगा कि अंतिम वर्गीकर्ता के वज़न प्रारंभ नहीं किए गए हैं, और मॉडल को पूर्व-प्रशिक्षण की आवश्यकता होगी - यह पूरी तरह से ठीक है, क्योंकि यही हम करने जा रहे हैं!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

अब हम प्रशिक्षण शुरू करने के लिए तैयार हैं! क्योंकि BERT पहले से ही प्री-ट्रेंड है, हम एक बहुत ही छोटे लर्निंग रेट से शुरुआत करना चाहते हैं ताकि प्रारंभिक वज़न खराब न हो जाएं।

सारा कठिन काम `BertForSequenceClassification` मॉडल द्वारा किया जाता है। जब हम प्रशिक्षण डेटा पर मॉडल को कॉल करते हैं, तो यह इनपुट मिनीबैच के लिए लॉस और नेटवर्क आउटपुट दोनों लौटाता है। हम पैरामीटर ऑप्टिमाइज़ेशन के लिए लॉस का उपयोग करते हैं (`loss.backward()` बैकवर्ड पास करता है), और `out` का उपयोग प्रशिक्षण सटीकता की गणना के लिए करते हैं, जो प्राप्त लेबल `labs` (जो `argmax` का उपयोग करके गणना किए जाते हैं) को अपेक्षित `labels` के साथ तुलना करता है।

प्रक्रिया को नियंत्रित करने के लिए, हम कई पुनरावृत्तियों में लॉस और सटीकता को संचित करते हैं, और हर `report_freq` प्रशिक्षण चक्रों के बाद उन्हें प्रिंट करते हैं।

यह प्रशिक्षण संभवतः काफी समय लेगा, इसलिए हम पुनरावृत्तियों की संख्या को सीमित करते हैं।


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


आप देख सकते हैं (खासकर अगर आप iterations की संख्या बढ़ाते हैं और थोड़ा अधिक इंतजार करते हैं) कि BERT classification हमें काफी अच्छी सटीकता देता है! इसका कारण यह है कि BERT पहले से ही भाषा की संरचना को काफी अच्छी तरह समझता है, और हमें केवल अंतिम classifier को fine-tune करना होता है। हालांकि, क्योंकि BERT एक बड़ा मॉडल है, पूरा training प्रक्रिया काफी समय लेती है और इसके लिए गंभीर computational power की आवश्यकता होती है! (GPU, और बेहतर होगा कि एक से अधिक GPU हों)।

> **Note:** हमारे उदाहरण में, हमने सबसे छोटे pre-trained BERT मॉडल्स में से एक का उपयोग किया है। बड़े मॉडल्स उपलब्ध हैं, जो संभवतः बेहतर परिणाम दे सकते हैं।


## मॉडल प्रदर्शन का मूल्यांकन

अब हम अपने मॉडल के प्रदर्शन का परीक्षण डेटा सेट पर मूल्यांकन कर सकते हैं। मूल्यांकन लूप प्रशिक्षण लूप के समान ही है, लेकिन हमें यह नहीं भूलना चाहिए कि मॉडल को मूल्यांकन मोड में स्विच करने के लिए `model.eval()` कॉल करना आवश्यक है।


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## मुख्य बातें

इस यूनिट में, हमने देखा कि **transformers** लाइब्रेरी से प्री-ट्रेंड भाषा मॉडल लेना और उसे हमारे टेक्स्ट क्लासिफिकेशन टास्क के लिए अनुकूलित करना कितना आसान है। इसी तरह, BERT मॉडल का उपयोग एंटिटी एक्सट्रैक्शन, प्रश्न उत्तर देने और अन्य NLP टास्क के लिए किया जा सकता है।

ट्रांसफॉर्मर मॉडल NLP में वर्तमान में सबसे उन्नत तकनीक का प्रतिनिधित्व करते हैं, और अधिकांश मामलों में, जब आप कस्टम NLP समाधान लागू करना शुरू करते हैं, तो यह पहला विकल्प होना चाहिए जिसके साथ आप प्रयोग करें। हालांकि, यदि आप उन्नत न्यूरल मॉडल बनाना चाहते हैं, तो इस मॉड्यूल में चर्चा किए गए पुनरावर्ती न्यूरल नेटवर्क के मूलभूत सिद्धांतों को समझना अत्यंत महत्वपूर्ण है।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
